In [1]:
from gwpy.timeseries import TimeSeries
import numpy as np
import matplotlib.pyplot as plt

from pycbc import psd
from pycbc.filter import highpass

In [2]:
def Whitening_strain(eval_psd, data):
        """
            PyCBC whitening. In this function we follow the procedure of 
            Tutorial 3.
        Input
        -----
        eval_psd: (pyCBC TimeSeries) data to compute PSD
        data: (pyCBC TimeSeries) data to whiten
        
        Output
        ------
        data_whitened: (pyCBC TimeSeries) whitened data
        """

        # We'll choose 4 seconds PSD samples that are overlapped 50 %
        seg_len = int(4 / eval_psd.delta_t)
        seg_stride = int(seg_len / 2)
        
        # We estimate the PSD with Welch's method
        PSD = psd.welch(eval_psd,
                    seg_len=seg_len,
                    seg_stride=seg_stride)
        # Interpolate PSD to adjust to data duration
        PSD = psd.interpolate(PSD, 1.0 / data.duration)
        PSD = psd.inverse_spectrum_truncation(PSD,
                                          int(4 * data.sample_rate),
                                          low_frequency_cutoff=30)
        # whiten
        data_whitened = (data.to_frequencyseries() / PSD ** 0.5)
        data_whitened = data_whitened.to_timeseries()

        return data_whitened


In [3]:
np.random.seed(1)
clean_segments = np.load('/home/robin.vanderlaag/wp5/strain_fractals/condor_data/clean_segments_O3A.npy')
msk = np.where((clean_segments[:,1]-clean_segments[:,0] >= 2*60))[0] # only segments longer than 2 minutes
clean_segments = clean_segments[msk]
n = 1 # number of clean times
clean_ids = np.random.choice(np.arange(clean_segments.shape[0]), n, replace=False)
clean = (clean_segments[clean_ids,0]+clean_segments[clean_ids,1])/2 # take the middle of each segment, should be furthest from any glitch/event

In [7]:
time = clean[0]
srate = 16384
length = 8
scratch = 20
duration = (scratch+length)/2
flow = 30.0
channel = 'L1:DCS-CALIB_STRAIN_C01'

eval_data = TimeSeries.get(channel=channel, start=time-duration*2, end=time)
data = TimeSeries.get(channel=channel, start=time-duration, end=time+duration)
data.shape[0]/srate

28.0

In [ ]:
plt.plot(eval_data, label='evaluate PSD')
plt.plot(data, label='data to whiten')
plt.xlabel('GPS time'), plt.ylabel('Amplitude')
plt.legend()
data.shape, eval_data.shape

In [ ]:
data, eval_data = data.to_pycbc(), eval_data.to_pycbc()
data, eval_data = highpass(data, flow), highpass(eval_data, flow)

In [ ]:
whiten_data = Whitening_strain(eval_data, data)
whiten_data = whiten_data[srate*10:-srate*10]
whiten_data.shape[0]/srate

In [ ]:
plt.plot(whiten_data)
plt.xlabel('GPS time'), plt.ylabel('Amplitude')

In [ ]:
plt.plot(whiten_data.psd(4))
plt.xscale('log'), plt.yscale('log')
plt.xlabel('Frequency (Hz)'), plt.ylabel('Amplitude')